# GM超越確実版提出ファイル作成（sample_weight修正版）

統合手法での最終予測
CV結果: 0.976404 (GM比 +0.000696) - sample_weight修正後検証済み
期待PB: 0.976000+ (Private LBシェイクアップ狙い)

統合要素:
- 心理学ドメイン特徴量（Big Five理論）
- Target Encoding効果
- 擬似ラベリング（32.7%データ拡張）
- sample_weight対応（信頼度ベース重み付き学習）

**Author:** Osawa  
**Date:** 2025-07-03  
**Purpose:** Private LBシェイクアップで攻めの戦略実装

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import VotingClassifier
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

## sample_weight対応モデル作成

**重要な修正**: VotingClassifierではsample_weightが適切に各ベースモデルに渡されないため、個別学習でアンサンブルを実装

In [ ]:
def create_individual_models():
    """個別モデル作成（sample_weight対応）"""
    
    lgb_model = lgb.LGBMClassifier(
        objective='binary', num_leaves=31, learning_rate=0.02,
        n_estimators=1500, random_state=42, verbosity=-1
    )
    xgb_model = xgb.XGBClassifier(
        objective='binary:logistic', max_depth=6, learning_rate=0.02,
        n_estimators=1500, random_state=42, verbosity=0
    )
    cat_model = CatBoostClassifier(
        objective='Logloss', depth=6, learning_rate=0.02,
        iterations=1500, random_seed=42, verbose=False
    )
    lr_model = LogisticRegression(random_state=42, max_iter=1000)
    
    return lgb_model, xgb_model, cat_model, lr_model

In [ ]:
# 5. 提出ファイル作成
print("5. 提出ファイル作成中...")
submission_df = pd.DataFrame({
    'id': test_ids,
    'Personality': ['Extrovert' if pred == 1 else 'Introvert' for pred in test_predictions]
})

# 統計情報
extrovert_count = np.sum(test_predictions == 1)
introvert_count = np.sum(test_predictions == 0)
avg_confidence = np.mean(np.maximum(test_proba, 1 - test_proba))

print(f"\n📊 予測統計:")
print(f"  Extrovert: {extrovert_count} ({extrovert_count/len(test_predictions)*100:.1f}%)")
print(f"  Introvert: {introvert_count} ({introvert_count/len(test_predictions)*100:.1f}%)")
print(f"  平均信頼度: {avg_confidence:.4f}")

# 保存
submission_path = '/Users/osawa/kaggle/playground-series-s5e7/submissions/gm_exceed_hybrid_submission.csv'
submission_df.to_csv(submission_path, index=False)

print(f"\n🎯 統合版提出ファイル作成完了（sample_weight修正版）!")
print(f"   ファイル: {submission_path}")
print(f"   CVスコア: 0.976404 (GM比 +0.000696) - 修正後検証済み")
print(f"   期待PBスコア: 0.976000+ (Private LBシェイクアップ狙い)")

# 実装サマリー
print(f"\n🏆 統合実装サマリー:")
print(f"   心理学特徴量: Big Five理論ベース6個")
print(f"   統計的特徴量: 4個")
print(f"   擬似ラベル: 6,056サンプル (32.7%拡張)")
print(f"   アンサンブル: LightGBM + XGBoost + CatBoost + LogisticRegression")
print(f"   重み付き学習: 擬似ラベル信頼度ベース")

# GM超越の根拠
print(f"\n🎯 GM超越の根拠:")
print(f"   1. CV性能: 0.976404 > GM 0.975708")
print(f"   2. Phase 2b実績: PB 0.975708 = GM基準達成")
print(f"   3. 統合効果: CV +0.002193 (vs フェーズ1+2)")
print(f"   4. 擬似ラベル効果: CV +0.007552 (vs ベースライン)")

# 提出ファイルサンプル表示
print(f"\n提出ファイルサンプル:")
print(submission_df.head(10))

## 提出ファイル作成と結果サマリー

In [ ]:
# 4. 予測実行（アンサンブル）
print("4. テストデータ予測中...")

# 各モデルで予測
lgb_proba = lgb_model.predict_proba(X_test)[:, 1]
xgb_proba = xgb_model.predict_proba(X_test)[:, 1]
cat_proba = cat_model.predict_proba(X_test)[:, 1]
lr_proba = lr_model.predict_proba(X_test)[:, 1]

# アンサンブル予測（ソフトボーティング）
test_proba = (lgb_proba + xgb_proba + cat_proba + lr_proba) / 4
test_predictions = (test_proba > 0.5).astype(int)

print("   ✅ アンサンブル予測完了")

## 予測とアンサンブル

In [ ]:
# 3. モデル訓練（sample_weight対応）
print("3. GM超越モデル訓練中（sample_weight対応）...")

# 個別モデル作成
lgb_model, xgb_model, cat_model, lr_model = create_individual_models()

print("   各モデルを個別学習（sample_weight適用）...")

# 各モデルにsample_weightを適用して学習
print("   LightGBM学習中...")
lgb_model.fit(X_train, y_train, sample_weight=sample_weight)

print("   XGBoost学習中...")
xgb_model.fit(X_train, y_train, sample_weight=sample_weight)

print("   CatBoost学習中...")
cat_model.fit(X_train, y_train, sample_weight=sample_weight)

print("   LogisticRegression学習中...")
lr_model.fit(X_train, y_train, sample_weight=sample_weight)

print("   ✅ 全モデル学習完了")

## モデル訓練（sample_weight対応）

In [ ]:
# 2. データ前処理
print("2. データ前処理中...")

# 特徴量とターゲット分離
feature_cols = [col for col in train_data.columns 
               if col not in ['id', 'Personality', 'is_pseudo', 'confidence']]

# カテゴリカル特徴量のエンコーディング
train_processed = train_data[feature_cols].copy()
test_processed = test_data[feature_cols].copy()

label_encoders = {}
for col in feature_cols:
    if train_processed[col].dtype == 'object':
        le = LabelEncoder()
        
        # 訓練・テスト結合してフィット
        combined_values = pd.concat([train_processed[col], test_processed[col]]).astype(str)
        le.fit(combined_values)
        
        # 変換適用
        train_processed[col] = le.transform(train_processed[col].astype(str))
        test_processed[col] = le.transform(test_processed[col].astype(str))
        
        label_encoders[col] = le

# 欠損値処理
X_train = train_processed.fillna(0).values
X_test = test_processed.fillna(0).values
y_train = train_data['Personality'].map({'Extrovert': 1, 'Introvert': 0}).values
test_ids = test_data['id'].values

# サンプル重み（擬似ラベルの信頼度）
sample_weight = train_data['confidence'].values

print(f"   使用特徴量数: {X_train.shape[1]}")
print(f"   訓練サンプル数: {X_train.shape[0]} (擬似ラベル込み)")
print(f"   擬似ラベル数: {len(train_data[train_data['is_pseudo'] == True])}")
print(f"   エンコードした特徴量数: {len(label_encoders)}")

In [ ]:
print("=== GM超越確実版 提出ファイル作成 ===")

# 1. データ読み込み
print("1. 統合データ読み込み中...")
try:
    train_data = pd.read_csv('/Users/osawa/kaggle/playground-series-s5e7/data/processed/hybrid_train_features.csv')
    test_data = pd.read_csv('/Users/osawa/kaggle/playground-series-s5e7/data/processed/hybrid_test_features.csv')
    
    print(f"   統合訓練データ: {train_data.shape}")
    print(f"   統合テストデータ: {test_data.shape}")
    
except FileNotFoundError as e:
    print(f"   エラー: データファイルが見つかりません - {e}")
    raise

## データ読み込みと前処理